# Eat Safe, Love

## Notebook Set Up

In [18]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [19]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [20]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [21]:
# review the collections in our database
db.list_collection_names()

['establishments']

In [31]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [32]:
# Find the establishments with a hygiene score of 20
query = {"scores.Hygiene": 20}

# Execute the query and retrieve matching documents
results = list(establishments.find(query))

# Print each matching document
for establishment in results:
    pprint(establishment)

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print(f"Number of establishments with a hygiene score of 20: {count}")

# Display the first document in the results using pprint
first_document = establishments.find_one(query)
pprint(first_document)

Number of establishments with a hygiene score of 20: 0
None


In [33]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print(f"Number of rows in DataFrame: {len(df)}")

# Display the first 10 rows of the DataFrame
print(df.head(10))

Number of rows in DataFrame: 0
Empty DataFrame
Columns: []
Index: []


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [34]:
# Find establishments with London as the Local Authority and RatingValue >= 4
query = {
    "LocalAuthorityName": {"$regex": "London", "$options": "i"},  # Case-insensitive search for 'London'
    "RatingValue": {"$gte": 4} 
}

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print(f"Number of establishments in London with a RatingValue >= 4: {count}")

# Display the first document in the results using pprint
first_document = establishments.find_one(query)
pprint(first_document)

Number of establishments in London with a RatingValue >= 4: 0
None


In [35]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(list(results))

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {len(df)}")

# Display the first 10 rows of the DataFrame
print(df.head(10))

Number of rows in the DataFrame: 0
Empty DataFrame
Columns: []
Index: []


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [36]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.49014200
longitude = 0.08384000

query = {
    "RatingValue": 5,
    "geocode.latitude": {"$gte": latitude - degree_search, "$lte": latitude + degree_search},
    "geocode.longitude": {"$gte": longitude - degree_search, "$lte": longitude + degree_search}
}
sort = [("scores.Hygiene", 1)]
limit = 5

# Print the results
results = list(establishments.find(query).sort(sort).limit(limit))

# Print the results
for establishment in results:
    pprint(establishment)

In [37]:
# Convert the results to a Pandas DataFrame
df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {len(df)}")

# Display the first 10 rows of the DataFrame
print(df.head(10))

Number of rows in the DataFrame: 0
Empty DataFrame
Columns: []
Index: []


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [38]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest
# Define the aggregation pipeline
pipeline = [
    # Match establishments with a hygiene score of 0
    {"$match": {"scores.Hygiene": 0}},
    
    # Group the matches by Local Authority
    {"$group": {
        "_id": "$LocalAuthorityName",  # Group by Local Authority
        "count": {"$sum": 1}  # Count the number of establishments in each group
    }},
    
    # Sort the results from highest to lowest by the count
    {"$sort": {"count": -1}}  # -1 for descending order
]

# Execute the pipeline
results = establishments.aggregate(pipeline)

# Convert the results to a list for further use
result_list = list(results)

# Print the number of documents in the result
print(f"Number of documents in the result: {len(result_list)}")

# Print the first 10 results
pprint(result_list[:10])

Number of documents in the result: 0
[]


In [39]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(result_list)

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {df.shape[0]}")

# Display the first 10 rows of the DataFrame
print(df.head(10))

Number of rows in the DataFrame: 0
Empty DataFrame
Columns: []
Index: []
